In [21]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import math
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from statsmodels.tsa.statespace.varmax import VARMAX

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_error 



In [28]:
data = pd.read_csv('C:\\Users\\user\\OneDrive\\Desktop\\Research Project\\Codes\\final_sentiment_data.csv')

In [17]:
data.keys()

Index(['datetime', 'sentiment_vader', 'positive_count', 'neutral_count',
       'negative_count', 'ratio_positive', 'sentiment_polarization',
       'net_sentiment', 'positive_to_total', 'negative_to_total', 'Close',
       'Volume BTC', 'Volume USDT', 'trade_count'],
      dtype='object')

## Process the data for machine learing model training

In [29]:
data.set_index('datetime', inplace=True)

In [30]:
data = data.loc['2021-01-01 00:00:00':'2022-05-31 00:00:00']

In [31]:
timelag = 2
for i in range(1, timelag+1):
    data[f'Close_{i}hr'] = data['Close'].shift(i)
    data[f'sentiment_lag_{i}hr'] = data['sentiment_polarization'].shift(i)
    data[f'trade_count_{i}hr'] = data['trade_count'].shift(i)
    data[f'Volume USDT_{i}hr'] = data['trade_count'].shift(i)
    data[f'Volume BTC_{i}hr'] = data['trade_count'].shift(i)



In [5]:
timelag = 2
for i in range(1, timelag+1):
    data[f'Close_{i}hr'] = data['sentiment_polarization'].shift(i)
    data[f'Close_lag_back_{i}hr'] = data['sentiment_polarization'].shift(-i)
    data[f'sentiment_lag_{i}hr'] = data['sentiment_polarization'].shift(i)
    data[f'sentiment_lag_back_{i}hr'] = data['sentiment_polarization'].shift(-i)
    data[f'trade_count_{i}hr'] = data['trade_count'].shift(i)
    data[f'trade_count_back_{i}hr'] = data['trade_count'].shift(-i)
    data[f'Volume USDT_{i}hr'] = data['trade_count'].shift(i)
    data[f'Volume USDT_back_{i}hr'] = data['trade_count'].shift(-i)
    data[f'Volume BTC_{i}hr'] = data['trade_count'].shift(i)
    data[f'Volume BTC_back_{i}hr'] = data['trade_count'].shift(-i)


In [32]:
data.shape

(12311, 23)

In [33]:
data.keys

<bound method NDFrame.keys of                      sentiment_vader  positive_count  neutral_count  \
datetime                                                              
2021-01-01 00:00:00         0.583991        0.051003       0.009387   
2021-01-01 01:00:00         0.439338        0.089818       0.023956   
2021-01-01 02:00:00         0.431446        0.082880       0.028589   
2021-01-01 03:00:00         0.438948        0.063191       0.023286   
2021-01-01 04:00:00         0.537289        0.063004       0.015788   
...                              ...             ...            ...   
2022-05-30 20:00:00         0.367085        0.106319       0.039073   
2022-05-30 21:00:00         0.357056        0.136134       0.051387   
2022-05-30 22:00:00         0.375190        0.125258       0.047607   
2022-05-30 23:00:00         0.389964        0.098631       0.033526   
2022-05-31 00:00:00         0.414391        0.097131       0.027979   

                     negative_count  ratio_pos

In [34]:
null_count = data.isnull().sum().sum()

# Print the result
print("Number of null values:", null_count)

Number of null values: 15


In [35]:
data.dropna(inplace=True)
data.drop(['ratio_positive', 'positive_count', 'neutral_count',	'negative_count', 'sentiment_vader', 'positive_to_total',	'net_sentiment', 'positive_to_total', 'negative_to_total'], axis = 1, inplace= True)

In [36]:
# Assume df is your DataFrame and 'Close' is your target column
target_column = 'Close'

# Compute the correlation of each numeric column with the target column
correlations = data.apply(lambda x: x.corr(data[target_column]))

# Drop the target column from the correlations
correlations = correlations.drop(target_column)

# Sort the correlation values in descending order
sorted_columns = correlations.sort_values(ascending=False).index

# Reorder the DataFrame columns based on the sorted correlation values
sorted_df = data[[target_column] + list(sorted_columns)]

In [37]:
sorted_df

,Close,Close_1hr,Close_2hr,sentiment_polarization,sentiment_lag_1hr,sentiment_lag_2hr,Volume USDT,trade_count_2hr,Volume USDT_2hr,Volume BTC_2hr,trade_count_1hr,Volume USDT_1hr,Volume BTC_1hr,trade_count,Volume BTC
datetime,,,,,,,,,,,,,,,
2021-01-01 02:00:00,0.224018,0.228255,0.220092,0.561453,0.534654,0.693206,0.046117,0.073059,0.073059,0.073059,0.129999,0.129999,0.129999,0.072129,0.053893
2021-01-01 03:00:00,0.225666,0.224018,0.228255,0.585239,0.561453,0.534654,0.028235,0.129999,0.129999,0.129999,0.072129,0.072129,0.072129,0.053190,0.033032
2021-01-01 04:00:00,0.224523,0.225666,0.224018,0.670205,0.585239,0.561453,0.039364,0.072129,0.072129,0.072129,0.053190,0.053190,0.053190,0.069336,0.046068
2021-01-01 05:00:00,0.223868,0.224523,0.225666,0.732950,0.670205,0.585239,0.028303,0.053190,0.053190,0.053190,0.069336,0.069336,0.069336,0.052302,0.033227
2021-01-01 06:00:00,0.223619,0.223868,0.224523,0.562987,0.732950,0.670205,0.027367,0.069336,0.069336,0.069336,0.052302,0.052302,0.052302,0.058058,0.032114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-30 20:00:00,0.264993,0.252517,0.253141,0.470314,0.617700,0.587794,0.275550,0.030513,0.030513,0.030513,0.049669,0.049669,0.049669,0.158561,0.304045
2022-05-30 21:00:00,0.275437,0.264993,0.252517,0.458503,0.470314,0.617700,0.265325,0.049669,0.049669,0.049669,0.158561,0.158561,0.158561,0.231511,0.287252
2022-05-30 22:00:00,0.274659,0.275437,0.264993,0.487817,0.458503,0.470314,0.131152,0.158561,0.158561,0.158561,0.231511,0.231511,0.231511,0.075123,0.142063


In [38]:
sorted_df.keys

<bound method NDFrame.keys of                         Close  Close_1hr  Close_2hr  sentiment_polarization  \
datetime                                                                      
2021-01-01 02:00:00  0.224018   0.228255   0.220092                0.561453   
2021-01-01 03:00:00  0.225666   0.224018   0.228255                0.585239   
2021-01-01 04:00:00  0.224523   0.225666   0.224018                0.670205   
2021-01-01 05:00:00  0.223868   0.224523   0.225666                0.732950   
2021-01-01 06:00:00  0.223619   0.223868   0.224523                0.562987   
...                       ...        ...        ...                     ...   
2022-05-30 20:00:00  0.264993   0.252517   0.253141                0.470314   
2022-05-30 21:00:00  0.275437   0.264993   0.252517                0.458503   
2022-05-30 22:00:00  0.274659   0.275437   0.264993                0.487817   
2022-05-30 23:00:00  0.273985   0.274659   0.275437                0.494755   
2022-05-31 00:00:00  0

In [39]:
sorted_df.to_csv('final_data_LSTM.csv')

## Spliting the data into training data and test data

In [40]:
test_split=round(len(sorted_df)*0.30)
test_split

3693

In [41]:
df_for_training=sorted_df[:-3692]
df_for_testing=sorted_df[-3692:]
print(df_for_training.shape)
print(df_for_testing.shape)

(8617, 15)
(3692, 15)


In [51]:
df_for_training

,Close,Close_1hr,Close_2hr,sentiment_polarization,sentiment_lag_1hr,sentiment_lag_2hr,Volume USDT,trade_count_2hr,Volume USDT_2hr,Volume BTC_2hr,trade_count_1hr,Volume USDT_1hr,Volume BTC_1hr,trade_count,Volume BTC
datetime,,,,,,,,,,,,,,,
2021-01-01 02:00:00,0.224018,0.228255,0.220092,0.561453,0.534654,0.693206,0.046117,0.073059,0.073059,0.073059,0.129999,0.129999,0.129999,0.072129,0.053893
2021-01-01 03:00:00,0.225666,0.224018,0.228255,0.585239,0.561453,0.534654,0.028235,0.129999,0.129999,0.129999,0.072129,0.072129,0.072129,0.053190,0.033032
2021-01-01 04:00:00,0.224523,0.225666,0.224018,0.670205,0.585239,0.561453,0.039364,0.072129,0.072129,0.072129,0.053190,0.053190,0.053190,0.069336,0.046068
2021-01-01 05:00:00,0.223868,0.224523,0.225666,0.732950,0.670205,0.585239,0.028303,0.053190,0.053190,0.053190,0.069336,0.069336,0.069336,0.052302,0.033227
2021-01-01 06:00:00,0.223619,0.223868,0.224523,0.562987,0.732950,0.670205,0.027367,0.069336,0.069336,0.069336,0.052302,0.052302,0.052302,0.058058,0.032114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27 05:00:00,0.650157,0.653078,0.653226,0.417951,0.488359,0.488066,0.025115,0.041486,0.041486,0.041486,0.042511,0.042511,0.042511,0.031349,0.016887
2021-12-27 06:00:00,0.651788,0.650157,0.653078,0.434106,0.417951,0.488359,0.027166,0.042511,0.042511,0.042511,0.031349,0.031349,0.031349,0.035480,0.018283
2021-12-27 07:00:00,0.647044,0.651788,0.650157,0.560692,0.434106,0.417951,0.032018,0.031349,0.031349,0.031349,0.035480,0.035480,0.035480,0.036356,0.021564


In [43]:
X_train = df_for_training.drop('Close', axis=1)

X_test = df_for_testing.drop('Close', axis=1)

y_train = df_for_training[['Close']]

y_test = df_for_testing[['Close']]

In [52]:
X_train

,Close_1hr,Close_2hr,sentiment_polarization,sentiment_lag_1hr,sentiment_lag_2hr,Volume USDT,trade_count_2hr,Volume USDT_2hr,Volume BTC_2hr,trade_count_1hr,Volume USDT_1hr,Volume BTC_1hr,trade_count,Volume BTC
datetime,,,,,,,,,,,,,,
2021-01-01 02:00:00,0.228255,0.220092,0.561453,0.534654,0.693206,0.046117,0.073059,0.073059,0.073059,0.129999,0.129999,0.129999,0.072129,0.053893
2021-01-01 03:00:00,0.224018,0.228255,0.585239,0.561453,0.534654,0.028235,0.129999,0.129999,0.129999,0.072129,0.072129,0.072129,0.053190,0.033032
2021-01-01 04:00:00,0.225666,0.224018,0.670205,0.585239,0.561453,0.039364,0.072129,0.072129,0.072129,0.053190,0.053190,0.053190,0.069336,0.046068
2021-01-01 05:00:00,0.224523,0.225666,0.732950,0.670205,0.585239,0.028303,0.053190,0.053190,0.053190,0.069336,0.069336,0.069336,0.052302,0.033227
2021-01-01 06:00:00,0.223868,0.224523,0.562987,0.732950,0.670205,0.027367,0.069336,0.069336,0.069336,0.052302,0.052302,0.052302,0.058058,0.032114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27 05:00:00,0.653078,0.653226,0.417951,0.488359,0.488066,0.025115,0.041486,0.041486,0.041486,0.042511,0.042511,0.042511,0.031349,0.016887
2021-12-27 06:00:00,0.650157,0.653078,0.434106,0.417951,0.488359,0.027166,0.042511,0.042511,0.042511,0.031349,0.031349,0.031349,0.035480,0.018283
2021-12-27 07:00:00,0.651788,0.650157,0.560692,0.434106,0.417951,0.032018,0.031349,0.031349,0.031349,0.035480,0.035480,0.035480,0.036356,0.021564


Linear Regression

In [44]:
steps = [
    ('poly', PolynomialFeatures()),  
    ('model', LinearRegression()) 
]
pipeline = Pipeline(steps)

# Define hyperparameters to tune
parameters = {
    'poly__degree': [1, 2, 3, 4],  # Degree of polynomial features
    'model__fit_intercept': [True, False],  # Whether to fit intercept
}

# Perform grid search cross-validation
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
test_r2 = r2_score(y_test, y_pred)
mae_score = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)

print('Best parameters:', grid_search.best_params_)
print('Best R-squared on training set:', grid_search.best_score_)
print('R-squared on test set:', test_r2)
print('Mean Absolute Error on test set:', mae_score)
print('Mean Squared Error on test set:', mse)
print('Root Mean Squared Error on test set:', rmse)

Best parameters: {'model__fit_intercept': True, 'poly__degree': 2}
Best R-squared on training set: -3.951137256666987e-05
R-squared on test set: 0.99824108457448
Mean Absolute Error on test set: 0.0029086644772265743
Mean Squared Error on test set: 1.729959763040907e-05
Root Mean Squared Error on test set: 0.004159278498779454


Testing the number of variables with the performance of linear regresion model as baseline

In [45]:
# Assuming X_train, X_test, y_train, y_test, and df_for_training are defined

for idx, column_name in enumerate(X_train.columns, 1):
    model = make_pipeline(PolynomialFeatures(degree=2), LinearRegression(fit_intercept=False))
    X_train_subset = X_train.iloc[:, :idx]
    X_test_subset = X_test.iloc[:, :idx]
    model.fit(X_train_subset, y_train)

    y_pred = model.predict(X_test_subset)
    test_r2 = r2_score(y_test, y_pred)
    mae_score = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse)
    
    print(f'Model performance with {idx} variables ({column_name})')
    print('R-squared on test set:', test_r2)
    print('Mean Squared Error on test set:', mse)
    print('--------------------------------------------')

Model performance with 1 variables (Close_1hr)
R-squared on test set: 0.9970736670226085
Mean Squared Error on test set: 2.8781590238488437e-05
--------------------------------------------
Model performance with 2 variables (Close_2hr)
R-squared on test set: 0.997084058348648
Mean Squared Error on test set: 2.867938762162387e-05
--------------------------------------------
Model performance with 3 variables (sentiment_polarization)
R-squared on test set: 0.9970857360880504
Mean Squared Error on test set: 2.8662886420845248e-05
--------------------------------------------
Model performance with 4 variables (sentiment_lag_1hr)
R-squared on test set: 0.9970861115973434
Mean Squared Error on test set: 2.8659193145102347e-05
--------------------------------------------
Model performance with 5 variables (sentiment_lag_2hr)
R-squared on test set: 0.9970900169180288
Mean Squared Error on test set: 2.8620782840947513e-05
--------------------------------------------
Model performance with 6 var

Decision tree

Testing the number of variables with the performance of descision tree model as baseline

In [46]:
for idx, column_name in enumerate(X_train.columns, 1):
    model = DecisionTreeRegressor(random_state=42)
    X_train_subset = X_train.iloc[:, :idx]
    X_test_subset = X_test.iloc[:, :idx]
    model.fit(X_train_subset, y_train)

    y_pred = model.predict(X_test_subset)
    test_r2 = r2_score(y_test, y_pred)
    mae_score = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse)
    
    print(f'Model performance with {idx} variables ({column_name})')
    print('R-squared on test set:', test_r2)
    print('Mean Squared Error on test set:', mse)
    print('--------------------------------------------')

Model performance with 1 variables (Close_1hr)
R-squared on test set: 0.9894219607962399
Mean Squared Error on test set: 0.00010403901136386366
--------------------------------------------
Model performance with 2 variables (Close_2hr)
R-squared on test set: 0.9904991942900986
Mean Squared Error on test set: 9.34440130328624e-05
--------------------------------------------
Model performance with 3 variables (sentiment_polarization)
R-squared on test set: 0.9871253586409573
Mean Squared Error on test set: 0.00012662696108962988
--------------------------------------------
Model performance with 4 variables (sentiment_lag_1hr)
R-squared on test set: 0.987696164643267
Mean Squared Error on test set: 0.00012101286843815219
--------------------------------------------
Model performance with 5 variables (sentiment_lag_2hr)
R-squared on test set: 0.9847346088143707
Mean Squared Error on test set: 0.00015014088872642535
--------------------------------------------
Model performance with 6 vari

In [47]:
# Define hyperparameters to tune
param_grid = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create Decision Tree regressor
tree_reg = DecisionTreeRegressor(random_state=42)

# Perform grid search cross-validation
grid_search = GridSearchCV(tree_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
test_r2 = r2_score(y_test, y_pred)
mae_score = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)


print('Best parameters:', grid_search.best_params_)
print('Best R-squared on training set:', grid_search.best_score_)
print('R-squared on test set:', test_r2)
print('Mean Absolute Error on test set:', mae_score)
print('Mean Squared Error on test set:', mse)
print('Root Mean Squared Error on test set:', rmse)

Best parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}
Best R-squared on training set: -0.00013538812418244616
R-squared on test set: 0.9930675006211179
Mean Absolute Error on test set: 0.005695296103219672
Mean Squared Error on test set: 6.818375010399902e-05
Root Mean Squared Error on test set: 0.008257345221316536


Random Forest

In [48]:
# Convert target variables to 1D arrays
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

# Define hyperparameters to tune
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create Random Forest regressor
rf_reg = RandomForestRegressor(random_state=42)

# Perform grid search cross-validation
grid_search = GridSearchCV(rf_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
test_r2 = r2_score(y_test, y_pred)
mae_score = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)


print('Best parameters:', grid_search.best_params_)
print('Best R-squared on training set:', grid_search.best_score_)
print('R-squared on test set:', test_r2)
print('Mean Absolute Error on test set:', mae_score)
print('Mean Squared Error on test set:', mse)
print('Root Mean Squared Error on test set:', rmse)

Best parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 150}
Best R-squared on training set: -0.00010739338649518826
R-squared on test set: 0.9960142182695167
Mean Absolute Error on test set: 0.004265745936620843
Mean Squared Error on test set: 3.9201668925959376e-05
Root Mean Squared Error on test set: 0.006261123615291378


XGBoost

In [49]:
regressor=xgb.XGBRegressor(tree_method="gpu_hist")

# set up our search grid
param_grid = {"max_depth":    [None, 5, 10, 15],
              "n_estimators": [500, 600, 700],
              "learning_rate": [0.01, 0.015]}

# try out every combination of the above values
search = GridSearchCV(regressor, param_grid, cv=5, scoring='neg_mean_squared_error').fit(X_train, y_train)

print("The best hyperparameters are ",search.best_params_)

The best hyperparameters are  {'learning_rate': 0.015, 'max_depth': 5, 'n_estimators': 600}


In [50]:
regressor=xgb.XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                           n_estimators  = search.best_params_["n_estimators"],
                           max_depth     = search.best_params_["max_depth"],
                           )

regressor.fit(X_train, y_train)

predictions = regressor.predict(X_test)

test_r2 = r2_score(y_test, predictions)
mae_score = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = math.sqrt(mse)
print('R-squared on test set:', test_r2)
print('Mean Absolute Error on test set:', mae_score)
print('Mean Squared Error on test set:', mse)
print('Root Mean Squared Error on test set:', rmse)

R-squared on test set: 0.9962876905832274
Mean Absolute Error on test set: 0.004100389962824424
Mean Squared Error on test set: 3.6511965417984904e-05
Root Mean Squared Error on test set: 0.006042513170691885
